In [3]:
#Парсер для цен ФИ
import yfinance as yf
import logging
import pandas as pd
import psycopg2
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from datetime import datetime, timedelta, date
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


logging.basicConfig(filename='update_prices.txt', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def create_connection(database_name, user, password, host, port):
    try:
        connection = psycopg2.connect(
            database=database_name,
            user=user,
            password=password,
            host=host,
            port=port
        )
        connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        return connection
    except Error as e:
        raise Exception(f"Ошибка при подключении к базе данных: {str(e)}")

def get_prices(start_date, end_date):
    success = False  # Флаг успешного обновления. Изначально считаем, что обновление неуспешно
    connection = None  # Инициализация переменной соединения
    connection = create_connection('db', 'user', 'password', 'localhost', '5435') #Данные для подключения
    cursor = connection.cursor()

    with open('tickers.txt', 'r') as file:
        tickers = [line.strip() for line in file]

    # Создание пустого DataFrame для хранения данных
    prices_df = pd.DataFrame()

    # Получение цен для каждого тикера
    for ticker in tickers:
        try:
            # Получение данных с помощью yfinance
            ticker_data = yf.Ticker(ticker)
            data = ticker_data.history(start=start_date, end=end_date)

            # Проверка наличия данных
            if not data.empty:
                data = data.round(2)
                data['Ticker'] = ticker
                prices_df = prices_df.append(data)

        except Exception as e:
            print(f"Ошибка при получении данных для тикера {ticker}: {e}")

    try:    
        if prices_df.empty:

            for ticker in tickers:
            # Получение предыдущих значений для каждого тикера
                query_prev_values = f"""
                    SELECT 
                        price_date,
                        COALESCE(Open, LAG(Open) OVER (PARTITION BY Ticker ORDER BY price_date)) AS Open,
                        COALESCE(High, LAG(High) OVER (PARTITION BY Ticker ORDER BY price_date)) AS High,
                        COALESCE(Low, LAG(Low) OVER (PARTITION BY Ticker ORDER BY price_date)) AS Low,
                        COALESCE(Close, LAG(Close) OVER (PARTITION BY Ticker ORDER BY price_date)) AS Close,
                        COALESCE(Volume, LAG(Volume) OVER (PARTITION BY Ticker ORDER BY price_date)) AS Volume,
                        COALESCE(Dividends, LAG(Dividends) OVER (PARTITION BY Ticker ORDER BY price_date)) AS Dividends,
                        COALESCE(Stock_Splits, LAG(Stock_Splits) OVER (PARTITION BY Ticker ORDER BY price_date)) AS Stock_Splits,
                        COALESCE(Capital_Gains, LAG(Capital_Gains) OVER (PARTITION BY Ticker ORDER BY price_date)) AS Capital_Gains,
                        Ticker
                    FROM asset_prices
                    WHERE Ticker = '{ticker}' AND price_date < '{start_date}'
                    ORDER BY price_date DESC
                    LIMIT 1
                """

                # Выполнение запроса для каждого тикера
                cursor.execute(query_prev_values)
                prev_values = cursor.fetchone()

                # Вставка данных
                query_insert = """
                INSERT INTO asset_prices
                (price_date, Open, High, Low, Close, Volume, Dividends, Stock_Splits, Capital_Gains, Ticker)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                cursor.execute(query_insert, (
                    start_date, prev_values[1], prev_values[2], prev_values[3],
                    prev_values[4], prev_values[5], prev_values[6], prev_values[7],
                    prev_values[8], ticker
                ))


        else:
            # Сброс индекса и удаление информации о часовом поясе
            prices_df = prices_df.reset_index()
            prices_df = prices_df.sort_values(by=['Date'])
            prices_df['Date'] = pd.to_datetime(prices_df['Date'], utc=True).dt.date

            prices_df = prices_df.rename(columns=lambda x: x.replace(' ', '_') if ' ' in x else x)
            prices_df = prices_df.rename(columns={'Date': 'price_date'})

            for _, row in prices_df.iterrows():
                query = """
                    INSERT INTO asset_prices
                    (price_date, Open, High, Low, Close, Volume, Dividends, Stock_Splits, Capital_Gains, Ticker)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
                    
                cursor.execute(query, (
                    row['price_date'], row['Open'], row['High'], row['Low'], row['Close'],
                    row['Volume'], row['Dividends'], row['Stock_Splits'], row['Capital_Gains'], row['Ticker']
                    ))

        connection.commit()
        cursor.close()
        success = True


    except Exception as e:
        # Обработка и логирование ошибок
        error_message = f"Произошла ошибка: {str(e)}"
        logging.error(error_message)
        print(error_message)

    finally:
        # Закрытие соединения с базой данных
        if connection:
            connection.close()

    # Логирование результатов
    if success:
        logging.info("Обновление базы данных завершено успешно!")
    else:
        logging.warning("Обновление базы данных завершилось с ошибкой!")


if __name__ == "__main__":
    start_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    end_date = (datetime.now()).strftime('%Y-%m-%d')
    prices_data = get_prices(start_date, end_date)
    
